In [124]:
import re
import geonamescache
import pandas as pd

from unidecode import unidecode

gc = geonamescache.GeonamesCache()

df = pd.DataFrame()

regexp = re.compile(r"[A-Za-z]+")

gc.get_cities_by_name("Miami Beach")

gc.get_cities_by_name("New York City")

headlines = []
countries = []
lats = []
longs =[]
places = []


def find_city(c_word, n_word, nn_word):
    
    c_city = None
    n_city = None
    nn_city = None
    cn_city = None  
    cnn_city = None
    
    #print(f"finding geo for city {c_word} {n_word} {nn_word}")
    
    #Three words
    search_word = c_word + " " + n_word + " " + nn_word
    #print(f"searching for: {search_word}")
    cnn_city = gc.get_cities_by_name(search_word)
    
    if len(cnn_city) > 0:
        #print(f"Searched for city {search_word} got geo: {cnn_city}")
        return (3,search_word,cnn_city)
    
    #first two words   
    search_word = c_word + " " + n_word
    #print(f"searching for: {search_word}")
    cn_city = gc.get_cities_by_name(search_word)
    
    if len(cn_city) > 0 :
        #print(f"searched for city {search_word} got geo: {cn_city}")
        return (2,search_word,cn_city)
    
    #last two words
    search_word = n_word + " " + nn_word
    #print(f"searching for: {search_word}")
    nn_city = gc.get_cities_by_name(search_word)
    if len(nn_city) > 0:
        #print(f"searched for city {search_word} got geo: {nn_city}")
        return (2,search_word,nn_city)
    
    #first word
    c_city = gc.get_cities_by_name(c_word)
    #print(f"searching for: {c_word}")

    if len(c_city) > 0:
        #print(f"serached for {c_word} geo: {c_city}")
        return (1,c_word,c_city)
    
    #second word
    n_city = gc.get_cities_by_name(n_word)
    #print(f"searching for: {n_word}")

    if len(n_city) > 0:
        #print(f"serached for {n_word} geo: {n_city}")
        return (1,n_word,n_city)
    
    #third word
    nn_city = gc.get_cities_by_name(nn_word)
    #print(f"searching for: {nn_word}")

    if len(nn_city) > 0:
        #print(f"serached for {nn_word} geo: {nn_city}")
        return (1,nn_word,nn_city)    
    
    
    return None



In [125]:
def data_to_dataframe(headline, geo_points):
    #print(geo_points)
    
    t_place = "None"
    t_country = "None"
    t_lat = "None"
    t_long = "None"
    
        
    max_point = 0
    for point in geo_points:
        if point[0] > max_point:
            t_place = point[1]
            max_point = point[0]
            loc_data =  point[2]
            for loc in loc_data:
                for key in loc:
                    t_country = loc[key]["countrycode"]
                    t_lat = loc[key]["latitude"]
                    t_long = loc[key]["longitude"]
                    
    #print(f"{headline} {t_place} {t_country} {t_lat} {t_long}")
    
    headlines.append(headline)
    places.append(t_place)
    countries.append(t_country)
    lats.append(t_lat)
    longs.append(t_long)
                    
    
    

In [126]:
def save_dataframe():
    data = {"headlines": headlines, "places": places, "countries": countries, "latittudes": lats, "longitudes" : longs }
    df = pd.DataFrame(data)
    return df

In [129]:
geo_collections = {}
file = open("data/headlines.txt", "r")
for number,line in enumerate(file.readlines()):
    head_line = unidecode(line)
    result = regexp.findall(head_line)
    if result ==  None:
        print("Wrong with Regexp construction")
    else:
        g_pos = []
        for idx,res in enumerate(result):
            if (idx < (len(result) - 2)):
                result_city = find_city(result[idx], result[idx+1], result[idx+2])
                if result_city is not None:
                    g_pos.append(result_city)
                    
        geo_collections[number] = g_pos
        data_to_dataframe(line,geo_collections[number])

                
file.close()

df = save_dataframe()
df.columns
df.head()

,headlines,places,countries,latittudes,longitudes
0,Zika Outbreak Hits Miami\n,Miami,US,25.7743,-80.1937
1,Could Zika Reach New York City?\n,New York City,US,40.7143,-74.006
2,First Case of Zika in Miami Beach\n,Miami Beach,US,25.7906,-80.13
3,"Mystery Virus Spreads in Recife, Brazil\n",Recife,BR,-8.05389,-34.8811
4,Dallas man comes down with case of Zika\n,Dallas,US,44.9193,-123.317
